라이브러리 임포트



In [ ]:
from typing import Annotated, Optional, Literal, List, Dict, Any
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
import enum
import os
from dotenv import load_dotenv
import uuid

# LangChain 및 기타 필요한 라이브러리 임포트
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.messages.base import BaseMessage

# LangGraph 관련 임포트
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# 기존 RAG 코드에서 가져온 컴포넌트
from langchain_upstage import UpstageDocumentParseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

# 랭퓨즈
from langfuse import Langfuse, get_client
from langfuse.langchain import CallbackHandler

# 스트리밍
import asyncio
from langchain_core.output_parsers import StrOutputParser

설정 및 초기화

In [ ]:
# .env 파일 로드 (필요한 경우)
load_dotenv()



유틸리티 함수

In [ ]:
# Format documents function
def format_docs(docs):
    """Format documents into a single string"""
    return "\n\n".join(doc.page_content for doc in docs)


# Initialize document loading and processing
def initialize_rag_components(file_path: str = "./test_modified.pdf"):
    """Initialize all components for RAG"""
    print("문서 로딩 중...")

    # Document loading
    loader = UpstageDocumentParseLoader(
        file_path,
        split="page",
        output_format="markdown",
        ocr="auto",
        coordinates=True,
    )
    docs = loader.load()
    print(f"문서 로딩 완료: {len(docs)} 페이지")

    # Document chunking
    print("문서 청킹 중...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
    docs_splitter = splitter.split_documents(docs)
    print(f"청킹 완료: {len(docs_splitter)} 청크")

    # Embedding model
    print("임베딩 모델 로딩 중...")
    # Change 'mps' to 'cuda' for NVIDIA GPUs or 'cpu' if you don't have GPU
    device = "cpu"  # 기본값으로 CPU 사용
    try:
        import torch

        if torch.cuda.is_available():
            device = "cuda"
        elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            device = "mps"
    except ImportError:
        pass

    print(f"사용 중인 디바이스: {device}")

    hf_embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large-instruct",
        model_kwargs={"device": device},
        encode_kwargs={"normalize_embeddings": True},
    )

    # Vector store
    print("벡터 스토어 생성 중...")
    vectorstore = FAISS.from_documents(
        documents=docs_splitter,
        embedding=hf_embeddings,
    )
    print("벡터 스토어 생성 완료")

    # Retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5},
    )

    # LLM - 🔥 스트리밍 활성화
    print("LLM 초기화 중...")
    llm = ChatAnthropic(
        model="claude-3-haiku-20240307",
        temperature=0,
        streaming=True,  # 🔥 스트리밍 활성화
    )
    print("초기화 완료!")

    return {
        "retriever": retriever,
        "llm": llm,
    }


# 전역 변수
rag_components = None
conversation_history = []


# 모델 클래스
# Define categories for query classification
class Category(enum.Enum):
    DOCUMENT = "document"  # 문서 관련 질문
    GENERAL = "general"  # 일반적인 질문
    GREETING = "greeting"  # 인사말


# Pydantic model for structured output
class QueryClassification(BaseModel):
    """사용자 쿼리 분류 모델"""

    category: Category = Field(
        description="쿼리를 카테고리화 하세요. DOCUMENT(문서 관련 질문) / GENERAL(일반적인 질문) / GREETING(인사) 중에 하나로 구분하세요."
    )
    reasoning: str = Field(description="왜 이 카테고리를 선택했는지 설명하세요.")


# Define the state structure
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    context: List[str]
    category: Optional[str]


# 🔥 스트리밍 헬퍼 함수 추가
async def stream_llm_response(llm, formatted_message):
    """LLM 응답을 스트리밍으로 출력하고 전체 응답 반환"""
    print("🤖 AI: ", end="", flush=True)

    # 체인 생성
    chain = llm | StrOutputParser()

    # 스트리밍 실행
    full_response = ""
    async for chunk in chain.astream(formatted_message):
        print(chunk, end="", flush=True)
        full_response += chunk

    print()  # 줄바꿈
    return full_response


# Router function for categorizing queries
def router(state: State) -> Dict[str, Any]:
    """사용자 쿼리를 카테고리로 분류하는 라우터"""
    print("쿼리 분류 중...")

    # Get the most recent user message
    user_message = state["messages"][-1].content

    # Create the router input
    router_input = f"""
    다음 사용자 쿼리를 분석하고 카테고리를 결정하세요.
    카테고리:
    - document: 문서 내용에 관한 질문 (예: "아주대학교에 대해 알려줘", "이 문서에서 중요한 내용은?")
    - general: 일반적인 질문으로, 문서와 관련이 없음 (예: "오늘 날씨 어때?", "파이썬이란?")
    - greeting: 인사말 (예: "안녕", "반가워", "뭐해?")
    
    쿼리: {user_message}
    """

    # Get LLM
    llm = rag_components["llm"]

    # Structured output with the classification model
    structured_llm = llm.with_structured_output(QueryClassification)

    # Get classification with Langfuse tracking
    classification = structured_llm.invoke(router_input)

    category = classification.category.value
    print(f"분류 결과: {category} (이유: {classification.reasoning})")

    return {"category": category}


# Conditional routing function
def route_by_category(state: State) -> Literal["document_qa", "general_qa", "greeting"]:
    """카테고리에 기반하여 다음 노드를 결정"""
    category = state.get("category", "").lower()

    if category == "document":
        return "document_qa"
    elif category == "general":
        return "general_qa"
    elif category == "greeting":
        return "greeting"
    else:
        # 기본값은 일반 질의응답
        return "general_qa"


# Define LangGraph nodes
def retrieve_documents(state: State) -> Dict[str, Any]:
    """문서에서 관련 내용 검색"""
    print("문서 검색 중...")

    # Get the most recent user message
    user_message = state["messages"][-1]

    # Retrieve documents
    retriever = rag_components["retriever"]
    docs = retriever.invoke(user_message.content)

    # Format documents
    formatted_docs = format_docs(docs)
    print(f"검색 완료: {len(docs)} 문서 찾음")

    # Return updated state
    return {"context": [formatted_docs]}


# 🔥 스트리밍 버전으로 수정
async def document_qa(state: State) -> Dict[str, Any]:
    """문서 기반 질의응답 - 스트리밍"""
    print("문서 기반 응답 생성 중...")
    context = state["context"][0] if state["context"] else "문서 정보 없음"
    user_message = state["messages"][-1].content

    # 이전 대화들은 별도로 히스토리 구성
    history_messages = state["messages"][:-1]
    formatted_history = ""
    for msg in history_messages:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        formatted_history += f"{role}: {msg.content}\n"

    # Create prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """너는 친절한 한국어 AI 비서야. 
        제공된 문서 내용(context)과 이전 대화 내용을 참고해서 질문에 답해.
        반드시 한국어로만 대답하고, 문서에 없는 내용은 대답하지 말고 모른다고 해.
        
        참고 문서:
        {context}
        
        이전 대화:
        {chat_history}
        """,
            ),
            ("user", "{user_input}"),
        ]
    )

    llm = rag_components["llm"]
    formatted_message = prompt.format_messages(
        context=context, chat_history=formatted_history, user_input=user_message
    )

    # 🔥 스트리밍 LLM 호출
    response_content = await stream_llm_response(llm, formatted_message)
    print("문서 기반 응답 생성 완료")

    return {"messages": [AIMessage(content=response_content)]}


# 🔥 스트리밍 버전으로 수정
async def general_qa(state: State) -> Dict[str, Any]:
    """일반 질의응답 - 스트리밍"""
    print("일반 응답 생성 중...")
    user_message = state["messages"][-1].content

    # 이전 대화들은 별도로 히스토리 구성
    history_messages = state["messages"][:-1]
    formatted_history = ""
    for msg in history_messages:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        formatted_history += f"{role}: {msg.content}\n"

    # Create prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """너는 친절한 한국어 AI 비서야. 
        사용자의 질문에 대해 간결하고 정확하게 답변해. 한국어로 대답해.
        
        이전 대화:
        {chat_history}
        """,
            ),
            ("user", "{user_input}"),
        ]
    )

    llm = rag_components["llm"]
    formatted_message = prompt.format_messages(
        user_input=user_message, chat_history=formatted_history
    )

    # 🔥 스트리밍 LLM 호출
    response_content = await stream_llm_response(llm, formatted_message)
    print("일반 응답 생성 완료")

    return {"messages": [AIMessage(content=response_content)]}


# 🔥 스트리밍 버전으로 수정
async def greeting(state: State) -> Dict[str, Any]:
    """인사말에 응답 - 스트리밍"""
    print("인사 응답 생성 중...")
    user_message = state["messages"][-1].content

    # 이전 대화들은 별도로 히스토리 구성
    history_messages = state["messages"][:-1]
    formatted_history = ""
    for msg in history_messages:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        formatted_history += f"{role}: {msg.content}\n"

    # Create prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """너는 친절한 한국어 AI 비서야.
        사용자의 인사에 친근하고 따뜻하게 응답해. 간결하게 한국어로 대답해.

        이전 대화:
        {chat_history}
        """,
            ),
            ("user", "{user_input}"),
        ]
    )

    llm = rag_components["llm"]
    formatted_message = prompt.format_messages(
        user_input=user_message, chat_history=formatted_history
    )

    # 🔥 스트리밍 LLM 호출
    response_content = await stream_llm_response(llm, formatted_message)
    print("인사 응답 생성 완료")

    return {"messages": [AIMessage(content=response_content)]}


# 🔥 비동기 그래프 실행 함수
async def run_graph_streaming(user_input: str):
    """스트리밍으로 그래프 실행"""
    global conversation_history

    user_message = HumanMessage(content=user_input)
    conversation_history.append(user_message)

    initial_state = {
        "messages": conversation_history,
        "context": [],
        "category": None,
    }

    # 스트리밍 그래프 빌드
    graph_builder = StateGraph(State)

    # 동기 노드들
    graph_builder.add_node("router", router)
    graph_builder.add_node("retrieve", retrieve_documents)

    # 🔥 비동기 스트리밍 노드들
    graph_builder.add_node("document_qa", document_qa)
    graph_builder.add_node("general_qa", general_qa)
    graph_builder.add_node("greeting", greeting)

    # 엣지 연결
    graph_builder.add_edge(START, "router")
    graph_builder.add_conditional_edges(
        "router",
        route_by_category,
        {
            "document_qa": "retrieve",
            "general_qa": "general_qa",
            "greeting": "greeting",
        },
    )
    graph_builder.add_edge("retrieve", "document_qa")
    graph_builder.add_edge("document_qa", END)
    graph_builder.add_edge("general_qa", END)
    graph_builder.add_edge("greeting", END)

    # 그래프 컴파일
    streaming_graph = graph_builder.compile()

    # 🔥 비동기 실행
    result = await streaming_graph.ainvoke(initial_state)

    # AI 응답을 히스토리에 추가
    if "messages" in result and len(result["messages"]) > 1:
        ai_msg = result["messages"][-1]
        if isinstance(ai_msg, AIMessage):
            conversation_history.append(ai_msg)
            return ai_msg.content

    return "응답을 생성할 수 없습니다."


# 🔥 비동기 채팅 인터페이스
async def interactive_chat_streaming():
    """스트리밍 채팅 인터페이스"""
    print("=" * 60)
    print("🤖 LangGraph 라우팅 RAG 챗봇 시작 (스트리밍 지원)")
    print("💡 '종료' 입력 시 대화를 끝냅니다.")
    print("💡 '히스토리' 입력 시 현재 대화 히스토리를 확인합니다.")
    print("=" * 60)

    try:
        while True:
            try:
                user_input = input("\n🙋 사용자: ").strip()

                if not user_input:
                    print("❗ 메시지를 입력해주세요.")
                    continue

                if user_input.lower() == "종료":
                    print("👋 채팅을 종료합니다!")
                    break

                if user_input.lower() == "히스토리":
                    print("\n=== 현재 대화 히스토리 ===")
                    for i, msg in enumerate(conversation_history):
                        msg_type = "사용자" if isinstance(msg, HumanMessage) else "AI"
                        content = (
                            msg.content[:100] + "..."
                            if len(msg.content) > 100
                            else msg.content
                        )
                        print(f"{i+1}. {msg_type}: {content}")
                    print(f"총 {len(conversation_history)}개 메시지")
                    print("=" * 30)
                    continue

                # 🔥 스트리밍으로 응답 생성
                await run_graph_streaming(user_input)

            except KeyboardInterrupt:
                print("\n👋 채팅을 종료합니다!")
                break

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback

        traceback.print_exc()


# 🔥 메인 실행 함수
async def main_streaming():
    """메인 실행 함수"""
    global rag_components, conversation_history

    try:
        # RAG 컴포넌트 초기화
        print("RAG 컴포넌트 초기화 중...")
        rag_components = initialize_rag_components()
        conversation_history = []

        print("🚀 스트리밍 RAG 챗봇 시작!")

        # 스트리밍 채팅 시작
        await interactive_chat_streaming()

    except Exception as e:
        print(f"초기화 중 오류 발생: {str(e)}")
        import traceback

        traceback.print_exc()


# 실행 방법
# Jupyter Notebook에서:
await main_streaming()

RAG 컴포넌트 초기화 중...
문서 로딩 중...
문서 로딩 완료: 40 페이지
문서 청킹 중...
청킹 완료: 70 청크
임베딩 모델 로딩 중...
사용 중인 디바이스: mps
벡터 스토어 생성 중...
벡터 스토어 생성 완료
LLM 초기화 중...
초기화 완료!
🚀 스트리밍 RAG 챗봇 시작!
🤖 LangGraph 라우팅 RAG 챗봇 시작 (스트리밍 지원)
💡 '종료' 입력 시 대화를 끝냅니다.
💡 '히스토리' 입력 시 현재 대화 히스토리를 확인합니다.
쿼리 분류 중...
분류 결과: document (이유: 이 쿼리는 아주대학교 공과대학에 대한 정보를 요청하고 있으므로 document 카테고리에 해당합니다.)
문서 검색 중...
검색 완료: 5 문서 찾음
문서 기반 응답 생성 중...
🤖 AI: 아주대학교 공과대학에는 다음과 같은 학과들이 있습니다:

- 첨단신소재공학과: 첨단 반도체/디스플레이 신소재, 첨단 에너지 신소재, 첨단 경량 신소재 등 3대 핵심 전략 분야를 중심으로 교육 및 연구를 강화하고 있습니다. 

- AI Lab(아주혁신대학): 기존 여러 학과를 통합하여 복수의 세부 특화전공으로 구성된 학부입니다. 사회 수요를 중심으로 세부 전공을 편성하고 학생들의 전공 선택권을 보장하는 특화된 교육 체제를 운영하고 있습니다.

- 프런티어과학학부: 물리학, 화학, 생명과학을 바탕으로 다양한 마이크로 전공을 선택할 수 있어 심화된 전공지식과 융합 능력을 겸비한 과학자를 양성하는 것을 목표로 합니다.

이 외에도 기계공학과, 산업공학과, 화학공학과 등 다양한 전통적인 공학 분야의 학과들이 있습니다. 아주대학교 공과대학은 첨단 기술 분야와 융합 교육에 주력하고 있습니다.
문서 기반 응답 생성 완료
👋 채팅을 종료합니다!
